In [34]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.by import By

In [35]:
driver = webdriver.Chrome()
to_location = "RUH"
url = "https://www.kayak.com/flights/EVN-RUH/2024-10-29/2024-11-05?sort=bestflight_a&attempt=2&lastms=1727730593363"

driver.get(url)
sleep(5)
#pop_window = "/html/body/div[6]/div/div[2]/div/div/div[1]/div/span[2]/button"
driver.find_element(By.XPATH, pop_window).click()

In [37]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="listWrapper"]/div/div[2]/div[2]/div[5]/div/div')))

elements = driver.find_elements(By.XPATH, '//*[@id="listWrapper"]/div/div[2]/div[2]')

# Iterate through the list of elements and perform actions
for element in elements:
    print(element.text)  # Print the text of each element
    # You can perform other actions like clicking, extracting attributes, etc.



Book once, earn twice
Book now to earn your airline miles on top of our rewards.
Share
12:35 pm – 8:20 pm
flydubai
1 stop
DXB
8h 45m
EVN
-
RUH
2:55 pm – 4:25 am
+1
Aegean Airlines
1 stop
ATH
12h 30m
RUH
-
EVN
Ad
1
0
$780
Lite
Expedia
View Deal
Best
Save
Share
9:00 pm – 3:15 am
+1
Wizz Air Abu Dhabi, Etihad Airways
1 stop
AUH
7h 15m
EVN
-
RUH
3:25 pm – 1:00 am
+1
flyadeal, FLYONE Armenia
1 stop
IST
8h 35m
RUH
-
EVN
0
0
$615
Economy Basic
Mytrip
View Deal
Cheapest
Save
Share
9:00 pm – 7:15 am
+1
Multiple airlines
2 stops
AUH, DMM
11h 15m
EVN
-
RUH
7:25 pm – 2:25 am
+2
Wizz Air, Wizz Air Malta
2 stops
BUD, FCO
30h 00m
RUH
-
EVN
0
0
$371
Economy Basic, Basic
Mytrip
View Deal
Book your flight to unlock more savings
Members can save up to 30% when they add a hotel to their flight.
$780
1 stop
Expedia
|
Ad
$780
Expedia
View Deal
Save
Share
9:00 pm – 6:15 am
+1
Multiple airlines
2 stops
AUH, DMM
10h 15m
EVN
-
RUH
3:25 pm – 1:00 am
+1
flyadeal, FLYONE Armenia
1 stop
IST
8h 35m
RUH
-
EVN
0
0
$42

In [38]:
flight_texts = [element.text.replace('\n', ' ') for element in elements]

data = '"""\n' + '\n'.join(flight_texts) + '\n"""'

print(data)

with open('flight_data.txt', 'w', encoding='utf-8') as file:
    file.write(data)

"""
Book once, earn twice Book now to earn your airline miles on top of our rewards. Share 12:35 pm – 8:20 pm flydubai 1 stop DXB 8h 45m EVN - RUH 2:55 pm – 4:25 am +1 Aegean Airlines 1 stop ATH 12h 30m RUH - EVN Ad 1 0 $780 Lite Expedia View Deal Best Save Share 9:00 pm – 3:15 am +1 Wizz Air Abu Dhabi, Etihad Airways 1 stop AUH 7h 15m EVN - RUH 3:25 pm – 1:00 am +1 flyadeal, FLYONE Armenia 1 stop IST 8h 35m RUH - EVN 0 0 $615 Economy Basic Mytrip View Deal Cheapest Save Share 9:00 pm – 7:15 am +1 Multiple airlines 2 stops AUH, DMM 11h 15m EVN - RUH 7:25 pm – 2:25 am +2 Wizz Air, Wizz Air Malta 2 stops BUD, FCO 30h 00m RUH - EVN 0 0 $371 Economy Basic, Basic Mytrip View Deal Book your flight to unlock more savings Members can save up to 30% when they add a hotel to their flight. $780 1 stop Expedia | Ad $780 Expedia View Deal Save Share 9:00 pm – 6:15 am +1 Multiple airlines 2 stops AUH, DMM 10h 15m EVN - RUH 3:25 pm – 1:00 am +1 flyadeal, FLYONE Armenia 1 stop IST 8h 35m RUH - EVN 0 0

In [61]:
import pandas as pd
import re

with open('flight_data.txt', 'r', encoding='utf-8') as file:
    data = file.read()

pattern = re.compile(
    r"(\d{1,2}:\d{2}\s(?:am|pm))\s–\s(\d{1,2}:\d{2}\s(?:am|pm)\s\+\d+)\s"
    r"([^0-9]+?)\s"
    r"(\d+\s(?:stops|stop))\s"
    r"([^0-9]+?)\s"
    r"(\d{1,2}h\s\d{1,2}m)\sEVN\s-\sRUH\s"
    r"(\d{1,2}:\d{2}\s(?:am|pm))\s–\s(\d{1,2}:\d{2}\s(?:am|pm)\s\+\d+)\s"
    r"([^0-9]+?)\s"
    r"(\d+\s(?:stops|stop))\s"
    r"([^0-9]+?)\s"
    r"(\d{1,2}h\s\d{1,2}m)\sRUH\s-\sEVN\s"
    r"\d+\s\d+\s\$"
    r"(\d+)\s"
    r"([^\d]+?)\s"
    r"(Mytrip)"
)

# Parse data from the text using the defined pattern
flights = []
for match in pattern.finditer(data):
    flights.append({
        'Departure Time': match.group(1),
        'Arrival Time': match.group(2),
        'Airline 1': match.group(3).strip(),
        'Stops 1': match.group(4),
        'Transit Points 1': match.group(5).strip(),
        'Duration 1': match.group(6),
        'Return Departure Time': match.group(7),
        'Return Arrival Time': match.group(8),
        'Airline 2': match.group(9).strip(),
        'Stops 2': match.group(10),
        'Transit Points 2': match.group(11).strip(),
        'Duration 2': match.group(12),
        'Price': match.group(13),
        'Cabin': match.group(14).strip(),
        'Booking Site': match.group(15)
    })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(flights)

# Display the DataFrame
df


,Departure Time,Arrival Time,Airline 1,Stops 1,Transit Points 1,Duration 1,Return Departure Time,Return Arrival Time,Airline 2,Stops 2,Transit Points 2,Duration 2,Price,Cabin,Booking Site
0,9:00 pm,3:15 am +1,"Wizz Air Abu Dhabi, Etihad Airways",1 stop,AUH,7h 15m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,615,Economy Basic,Mytrip
1,9:00 pm,7:15 am +1,Multiple airlines,2 stops,"AUH, DMM",11h 15m,7:25 pm,2:25 am +2,"Wizz Air, Wizz Air Malta",2 stops,"BUD, FCO",30h 00m,371,"Economy Basic, Basic",Mytrip
2,9:00 pm,6:15 am +1,Multiple airlines,2 stops,"AUH, DMM",10h 15m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,429,Economy Basic,Mytrip
3,9:00 pm,7:15 am +1,Multiple airlines,2 stops,"AUH, DMM",11h 15m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,419,Economy Basic,Mytrip
4,9:00 pm,8:10 am +1,Multiple airlines,2 stops,"AUH, DMM",12h 10m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,419,Economy Basic,Mytrip
5,5:20 pm,3:15 am +1,"Wizz Air Abu Dhabi, Etihad Airways",1 stop,AUH,10h 55m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,615,Economy Basic,Mytrip
6,9:00 pm,8:15 am +1,Multiple airlines,2 stops,"AUH, DMM",12h 15m,3:25 pm,1:00 am +1,"flyadeal, FLYONE Armenia",1 stop,IST,8h 35m,429,Economy Basic,Mytrip


In [62]:
print(df.columns)

Index(['Departure Time', 'Arrival Time', 'Airline 1', 'Stops 1',
       'Transit Points 1', 'Duration 1', 'Return Departure Time',
       'Return Arrival Time', 'Airline 2', 'Stops 2', 'Transit Points 2',
       'Duration 2', 'Price', 'Cabin', 'Booking Site'],
      dtype='object')


# Summary

Summary of Findings:
1. Flight Route and Price Analysis:

The dataset includes information on multiple airlines, with flights predominantly involving one or two stops. Notably, Abu Dhabi (AUH) and Istanbul (IST) are common transit points, highlighting their roles as pivotal secondary airports in the route structure.
Flight durations vary, with outbound legs generally shorter than the inbound legs due to the number of stops and layover durations. The shortest direct flight duration noted is approximately 7 hours and 15 minutes, and the longest goes up to 30 hours, indicating significant variation based on route and airline.
Prices range from $371 to $615 for Economy Basic, reflecting a considerable variance that could be influenced by factors such as the number of stops, airline choice, and travel class.